<a href="https://colab.research.google.com/github/kartikgandhi/Soft-Voting-Ensemble-Results-Edited-Dataset/blob/main/Ensemble_model_Maven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from numpy import mean
from numpy import std

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [67]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [68]:
DATA = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-maven 3.3.3-3.3.9 - maven 3.3.3-3.3.9.csv")


In [69]:
DATA['Change'] = DATA['Change'].map({'yes': 1, 'no': 0})

In [70]:
DATA.shape

(831, 8)

In [71]:
DATA.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,22,10,26,237,1,73,13,0
1,2,0,18,40,0,40,5,0
2,4,0,24,71,0,27,11,0
3,2,2,16,15,0,0,3,0
4,3,0,14,7,0,0,1,0


In [72]:
X = DATA.drop('Change', axis=1)
y = DATA['Change']

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [74]:
# get a voting ensemble of models
def get_voting():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))

	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='soft')
	return ensemble

In [75]:
# get a list of models to evaluate
def get_models():
	models = dict()
	models['hard_voting'] = get_voting()
	return models

In [76]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [77]:
# get the models to evaluate
models = get_models()

In [78]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))

>hard_voting 0.204 (0.403)


In [79]:
print(y_pred)

[0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [80]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

[[104   8]
 [ 29  26]]
              precision    recall  f1-score   support

           0       0.78      0.93      0.85       112
           1       0.76      0.47      0.58        55

    accuracy                           0.78       167
   macro avg       0.77      0.70      0.72       167
weighted avg       0.78      0.78      0.76       167



In [81]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.8534090909090911


In [82]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.4683740565827546


In [83]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.7006493506493506


In [84]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.6625413488689131
